In [4]:
#from datetime import timedelta, datetime
#import time
import psycopg2
import pandas
from sklearn.cluster import KMeans


In [103]:
# Get the average temperature for each city per quarter
connection = psycopg2.connect(f"host='localhost' dbname='db' user='etl' password='etl'")
cursor = connection.cursor()
    
# Get the date of the last temperature loaded
sql = """select region, country, city, quarter , round( avg(avgtemp),2) as avgtemp
from temperatures
group by region, country, city , quarter 
order by region, country, city, quarter ;
"""

data = pandas.read_sql(sql, connection)

data.info()

# Close database connection
connection.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   region   1284 non-null   object 
 1   country  1284 non-null   object 
 2   city     1284 non-null   object 
 3   quarter  1284 non-null   int64  
 4   avgtemp  1284 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 50.3+ KB


In [104]:
# Set clusters based on temperature and quarter
model = KMeans(n_clusters=5, random_state=42).fit(data.drop(columns=["region","country","city"]))
pred = model.labels_
data["cluster"] = pred
data

,region,country,city,quarter,avgtemp,cluster
0,Africa,Algeria,Algiers,1,11.89,0
1,Africa,Algeria,Algiers,2,19.12,3
2,Africa,Algeria,Algiers,3,25.28,1
3,Africa,Algeria,Algiers,4,15.72,0
4,Africa,Benin,Cotonou,1,28.60,1
...,...,...,...,...,...,...
1279,South/Central America & Carribean,Uruguay,Montvideo,4,18.38,3
1280,South/Central America & Carribean,Venezuela,Caracas,1,26.13,1
1281,South/Central America & Carribean,Venezuela,Caracas,2,27.44,1
1282,South/Central America & Carribean,Venezuela,Caracas,3,28.13,1


In [112]:
cluster = 4
print(f"""
count: {data[data.cluster == cluster]["avgtemp"].count()}

min: {data[data.cluster == cluster]["avgtemp"].min()}

mean: {data[data.cluster == cluster]["avgtemp"].mean()}

max: {data[data.cluster == cluster]["avgtemp"].max()}
""")


count: 108

min: -17.29

mean: -2.1354629629629627

max: 1.94



In [106]:
# Get average temperature per cluster
clusterMetadata = pandas.DataFrame({
    "cluster": [0,1,2,3,4],
    "avgtemp":[data[data.cluster==0]["avgtemp"].mean(),
data[data.cluster==1]["avgtemp"].mean(),
data[data.cluster==2]["avgtemp"].mean(),
data[data.cluster==3]["avgtemp"].mean(),
data[data.cluster==4]["avgtemp"].mean()]})

# Set temperature level for each cluster
clusterMetadata = clusterMetadata.sort_values(by='avgtemp')
clusterMetadata['templevel'] = ['very low','low','medium','high','very high']

clusterMetadata


,cluster,avgtemp,templevel
4,4,-2.135463,very low
2,2,5.739231,low
0,0,13.993581,medium
3,3,20.839313,high
1,1,27.384949,very high


In [107]:
# Add level to main table
def setTempLevel(cluster):
    level = clusterMetadata[clusterMetadata.cluster == cluster]['templevel'].iloc[0]
    return str(level)
data['templevel'] = data.apply(lambda x : setTempLevel(x['cluster']) , axis=1)
data


,region,country,city,quarter,avgtemp,cluster,templevel
0,Africa,Algeria,Algiers,1,11.89,0,medium
1,Africa,Algeria,Algiers,2,19.12,3,high
2,Africa,Algeria,Algiers,3,25.28,1,very high
3,Africa,Algeria,Algiers,4,15.72,0,medium
4,Africa,Benin,Cotonou,1,28.60,1,very high
...,...,...,...,...,...,...,...
1279,South/Central America & Carribean,Uruguay,Montvideo,4,18.38,3,high
1280,South/Central America & Carribean,Venezuela,Caracas,1,26.13,1,very high
1281,South/Central America & Carribean,Venezuela,Caracas,2,27.44,1,very high
1282,South/Central America & Carribean,Venezuela,Caracas,3,28.13,1,very high


In [113]:
# Remove cluster column
data = data.drop(columns="cluster")
data

,region,country,city,quarter,avgtemp,templevel
0,Africa,Algeria,Algiers,1,11.89,medium
1,Africa,Algeria,Algiers,2,19.12,high
2,Africa,Algeria,Algiers,3,25.28,very high
3,Africa,Algeria,Algiers,4,15.72,medium
4,Africa,Benin,Cotonou,1,28.60,very high
...,...,...,...,...,...,...
1279,South/Central America & Carribean,Uruguay,Montvideo,4,18.38,high
1280,South/Central America & Carribean,Venezuela,Caracas,1,26.13,very high
1281,South/Central America & Carribean,Venezuela,Caracas,2,27.44,very high
1282,South/Central America & Carribean,Venezuela,Caracas,3,28.13,very high
